# アンサンブル学習のテスト
担当：梁木  
日付：2019/1/22

## 内容
アンサンブル学習をする上でのクラス作成  
train multiが未実装

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_iris
from multiprocessing import Pool
import multiprocessing as multi
from multiprocessing import Process
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
import statistics

## データの読み込み
sklearn付属のirisデータセットを読みこむ

In [2]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### 乱数でtrainデータとtestデータに分解

In [3]:
np.random.seed(seed=0)
test_indexs = np.random.choice(range(len(X)),20,replace=False)
test_indexs.sort()

X_train = X[X.index.isin(test_indexs) == False]
X_test = X[X.index.isin(test_indexs) == True]
y_train = y[np.isin(range(len(y)), test_indexs) == False]
y_test = y[test_indexs]

## EnsembleModelを定義
複数のモデルをまとめて学習し、それぞれの予測結果の最頻値または平均値を出力する

In [4]:
class EnsembleModel:
    def __init__(self, clfs, params, seed=0):
        for i in range(len(params)):
            params[i]['random_state'] = seed
        self.clfs = [clf(**param) for clf, param in zip(clfs, params)]
        
    def train(self, x_train, y_train, multi=False):
        self.x_train = x_train
        self.y_train = y_train
        if multi:
            with Pool(processes=2) as pool:
                pool.map(self._train, range(len(self.clfs)))
        else:
            for i in range(len(self.clfs)):
                self._train(i)
        
    def _train(self, i):
        self.clfs[i] = self.clfs[i].fit(self.x_train, self.y_train)
        
    def predict(self, x, en_type='mode'):
        if en_type == 'mode':
            return np.array([np.argmax(np.bincount(p)) for p in np.array([clf.predict(x) for clf in self.clfs]).T])
        if en_type == 'mean':
            return np.array([clf.predict(x) for clf in self.clfs]).mean(axis=0)

### 5つのモデルを使う

In [5]:
en_model = EnsembleModel([
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    ExtraTreesClassifier,
    SVC
], params=[{} for i in range(5)])

### モデルを学習

In [6]:
en_model.train(X_train, y_train)

### 学習したモデルで推論

In [7]:
en_model.predict(X_test)

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2])